In [1]:
import torch
from torch import nn
import torchvision
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets,transforms
from torchvision.transforms import ToTensor,InterpolationMode
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import random
import requests
import pandas as pd
from pathlib import Path
import numpy as np
import os
import zipfile
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, balanced_accuracy_score, classification_report
from tqdm.auto import tqdm
from collections import Counter

In [2]:
# set device by checking if gpu is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [3]:
# set seed for reproducibilty
SEED = 50

random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
# # mount google drive
# from google.colab import drive
# drive.mount('/content/drive')

# # root directory where the food-11 zip file is located
# root_dir = 'drive/MyDrive/'

**DATA PREPROCESSING AND VISUALIZATION**

In [5]:
# extract the food-11 zip file
with zipfile.ZipFile('google_stock_price.zip', 'r') as zip_ref:
    zip_ref.extractall('google_stock_price/')

In [6]:
df = pd.read_csv('google_stock_price/Google_Stock_Price_Train.csv')
df.head(10)

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"
5,1/10/2012,313.70,315.72,307.30,621.43,"8,824,000"
6,1/11/2012,310.59,313.52,309.40,624.25,"4,817,800"
7,1/12/2012,314.43,315.26,312.08,627.92,"3,764,400"
8,1/13/2012,311.96,312.30,309.37,623.28,"4,631,800"
9,1/17/2012,314.81,314.81,311.67,626.86,"3,832,800"


In [7]:
class StockDataset(Dataset):
    def __init__(self, data, N):
        data['Volume'] = data['Volume'].str.replace(',', '')

        self.data = data[['Open', 'High', 'Low', 'Close', 'Volume']].apply(pd.to_numeric, errors='coerce')
        self.data.fillna(0, inplace=True)

        self.N = N

    def __len__(self):
        return len(self.data) - self.N

    def __getitem__(self, idx):
        # Get the sequence of features for the last M days
        x = self.data.iloc[idx:idx + self.N, :].values  # Exclude the last column 'Close'
        # Get the closing price of the next day as the target
        y = self.data.iloc[idx + self.N]['Close']

        # Convert to PyTorch tensors
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)

        return x, y

In [8]:
# Define your parameters
N = 30  # Number of days to look back
batch_size = 32  # Batch size for DataLoader

# Instantiate the dataset
train_data = StockDataset(df, N)

In [9]:
# Create the DataLoader
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

In [10]:
x,y = next(iter(train_dataloader))

print(x.shape,y.shape)

torch.Size([32, 30, 5]) torch.Size([32])
